<a href="https://colab.research.google.com/github/CalvHobbes/mountain-classifier/blob/main/ViT_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers pip comet_ml datasets evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.1/698.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 15.0 MB/s eta 0:00:00
  Created wheel for configobj: filename=configobj-5.0.9-py2.py3-none-any.whl size=35615 sha256=938c590bd932c1f43bc25115615ddee46d1d2aa4bcebf64f046f300c7ab41f35
  Stored in directory: /root/.cache/pip/wheels/a1/6c/03/6c5e3cf1a6e4b9e2fc5c4409be4abc5a8268bd9c878739cb32
Successfully built configobj
  Attempting uninstall: python-box
    Found existing installation: python-box 7.2.0
    Uninstalling python-box-7.2.0:
      Successfully uninstalled python-box-7.2.0

In [3]:
!git clone https://github.com/CalvHobbes/mountain-classifier.git

Cloning into 'mountain-classifier'...
remote: Enumerating objects: 711, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 711 (delta 0), reused 3 (delta 0), pack-reused 705 (from 1)
Receiving objects: 100% (711/711), 228.64 MiB | 44.91 MiB/s, done.
Updating files: 100% (680/680), done.


In [5]:
from PIL import Image
import numpy as np
import torch

def preprocess_image(image):
    image = Image.open(image["image"]).convert("RGB")
    image = image.resize((224, 224))
    image = np.array(image) / 255.0
    image = image.transpose(2, 0, 1)  # Convert to (C, H, W) format
    image = torch.tensor(image)
    return image

In [1]:
from datasets import load_dataset

# Load dataset from directory structure (like `train` and `test` folders)
# This assumes your `train_dir` and `test_dir` folders have subfolders per class
train_dir = "mountain-classifier/data/train"
test_dir = "mountain-classifier/data/test"

# Load the training and test datasets
train_dataset = load_dataset("imagefolder", data_dir=train_dir)
test_dataset = load_dataset("imagefolder", data_dir=test_dir)



Resolving data files:   0%|          | 0/541 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/136 [00:00<?, ?it/s]

In [6]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 541
    })
})

In [7]:
test_dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 136
    })
})

In [2]:
from transformers import ViTFeatureExtractor,AutoModelForImageClassification

model_name_or_path = 'google/vit-base-patch16-224'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)
feature_extractor

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [6]:
def process_example(example):
    inputs = feature_extractor(example['image'], return_tensors='pt')
    inputs['labels'] = example['label']
    return inputs

In [7]:
process_example(train_dataset['train'][0])

{'pixel_values': tensor([[[[-0.9216, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137],
          [-0.9137, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137],
          [-0.9137, -0.9216, -0.9216,  ..., -0.9137, -0.9059, -0.9059],
          ...,
          [-0.1294, -0.0275,  0.0431,  ..., -0.6863, -0.7020, -0.7255],
          [-0.3412, -0.2549, -0.1216,  ..., -0.6000, -0.6706, -0.6627],
          [-0.3882, -0.3098, -0.2941,  ..., -0.4588, -0.6471, -0.7020]],

         [[-0.7255, -0.7176, -0.7176,  ..., -0.7725, -0.7725, -0.7725],
          [-0.7176, -0.7176, -0.7176,  ..., -0.7725, -0.7725, -0.7725],
          [-0.7176, -0.7255, -0.7255,  ..., -0.7725, -0.7647, -0.7647],
          ...,
          [-0.1686, -0.0902,  0.0118,  ..., -0.7255, -0.7569, -0.7804],
          [-0.3725, -0.3098, -0.1451,  ..., -0.6549, -0.7333, -0.7333],
          [-0.4275, -0.3412, -0.3255,  ..., -0.5216, -0.7255, -0.7804]],

         [[-0.2863, -0.2784, -0.2784,  ..., -0.3882, -0.3961, -0.3961],
          [-0

In [25]:
from PIL import Image
import torchvision.transforms as transforms

def transform(example_batch):
    # Define the desired image size
    desired_size = (224, 224)

    # Resize the images to the desired size
    resized_images = [transforms.Resize(desired_size)(x.convert("RGB")) for x in example_batch['image']]

    # Convert resized images to pixel values
    inputs = feature_extractor(resized_images, return_tensors='pt')

    # Don't forget to include the labels!
    inputs['label'] = example_batch['label']

    return inputs

prepared_train_ds = train_dataset.with_transform(transform)
prepared_test_ds = test_dataset.with_transform(transform)

In [31]:
prepared_train_ds['train'][0]["pixel_values"].shape

torch.Size([3, 224, 224])

In [12]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [32]:
labels = prepared_train_ds['train'].features['label'].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label


In [34]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([14]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([14, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
from transformers import Trainer, TrainingArguments
import comet_ml
from comet_ml import Experiment
from comet_ml import start
from comet_ml.integration.pytorch import log_model
from google.colab import userdata
import os

experiment = start(
  api_key="tv9OhbrvWqXeyrnIlTKVBUkwQ",
  project_name="mountain-classifier",
  workspace="calvhobbes"
)

comet_key = userdata.get("COMET_API_KEY")
comet_ml.login(project_name="mountain-classifier", api_key=comet_key)

os.environ["COMET_LOG_ASSETS"] = "True"
os.environ["COMET_API_KEY"] = comet_key

# training_args = TrainingArguments(
#     output_dir="my_awesome_food_model",
#     remove_unused_columns=False,
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=5e-5,
#     per_device_train_batch_size=16,
#     gradient_accumulation_steps=4,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     warmup_ratio=0.1,
#     logging_steps=10,
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy",
#     push_to_hub=True,
# )

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    fp16=True,
    logging_steps=10,
    save_total_limit=1,
    remove_unused_columns=False,
    push_to_hub=False,
    load_best_model_at_end=True,
    can_return_loss=True,
    metric_for_best_model="accuracy"
)
experiment.log_parameters(training_args)





COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key


In [41]:
import numpy as np
import evaluate

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

metric = evaluate.load("accuracy")

def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)


# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=prepared_train_ds['train'],
    eval_dataset=prepared_test_ds['train'],
    data_collator=collate_fn,
    compute_metrics=compute_metrics
)



/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [42]:
train_results = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.399759,0.198529
2,No log,2.229112,0.316176
3,No log,2.118124,0.345588
4,No log,2.058292,0.360294
5,No log,2.039594,0.389706


In [ ]:
log_model(experiment, model=model, model_name="Vit-14Summit-Classifier")

In [ ]:
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

In [ ]:
metrics = trainer.evaluate(prepared_train_ds['train'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)